Based on https://www.pinecone.io/learn/series/faiss/vector-indexes/ and https://www.pinecone.io/learn/series/faiss/product-quantization/

# Set up runtime

In [1]:
%pip install faiss-cpu
%pip install humanize

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [23]:
import os
import humanize
import shutil
import urllib.request as request
from contextlib import closing
import numpy
numpy.set_printoptions(threshold=numpy.inf)

# Load and prepare demo data

In [2]:
# first we download the Sift1M dataset
with closing(request.urlopen('ftp://ftp.irisa.fr/local/texmex/corpus/sift.tar.gz')) as r:
    with open('sift.tar.gz', 'wb') as f:
        shutil.copyfileobj(r, f)

In [2]:
import tarfile

# the download leaves us with a tar.gz file, we unzip it
tar = tarfile.open('sift.tar.gz', "r:gz")
tar.extractall()

In [2]:
import numpy as np

# now define a function to read the fvecs file format of Sift1M dataset
def read_fvecs(fp):
    a = np.fromfile(fp, dtype='int32')
    d = a[0]
    return a.reshape(-1, d + 1)[:, 1:].copy().view('float32')

In [3]:
# data we will search through
xb = read_fvecs('./sift/sift_base.fvecs')  # 1M samples
# also get some query vectors to search with
xq = read_fvecs('./sift/sift_query.fvecs')
# take just one query (there are many in sift_learn.fvecs)
xq = xq[0].reshape(1, xq.shape[1])

In [4]:
# The query vector
xq.shape

(1, 128)

In [5]:
# The vector search space
xb.shape

(1000000, 128)

In [25]:
# Sample of the vector embeddings
xb[:5]

array([[  0.,  16.,  35.,   5.,  32.,  31.,  14.,  10.,  11.,  78.,  55.,
         10.,  45.,  83.,  11.,   6.,  14.,  57., 102.,  75.,  20.,   8.,
          3.,   5.,  67.,  17.,  19.,  26.,   5.,   0.,   1.,  22.,  60.,
         26.,   7.,   1.,  18.,  22.,  84.,  53.,  85., 119., 119.,   4.,
         24.,  18.,   7.,   7.,   1.,  81., 106., 102.,  72.,  30.,   6.,
          0.,   9.,   1.,   9., 119.,  72.,   1.,   4.,  33., 119.,  29.,
          6.,   1.,   0.,   1.,  14.,  52., 119.,  30.,   3.,   0.,   0.,
         55.,  92., 111.,   2.,   5.,   4.,   9.,  22.,  89.,  96.,  14.,
          1.,   0.,   1.,  82.,  59.,  16.,  20.,   5.,  25.,  14.,  11.,
          4.,   0.,   0.,   1.,  26.,  47.,  23.,   4.,   0.,   0.,   4.,
         38.,  83.,  30.,  14.,   9.,   4.,   9.,  17.,  23.,  41.,   0.,
          0.,   2.,   8.,  19.,  25.,  23.,   1.],
       [ 14.,  35.,  19.,  20.,   3.,   1.,  13.,  11.,  16., 119.,  85.,
          5.,   0.,   5.,  24.,  26.,   0.,  27., 119.,  13.,

# Flat Index

In [6]:
D = xb.shape[1]  # dimensionality of Sift1M data
k = 100  # number of nearest neighbors to return

import faiss

index = faiss.IndexFlatIP(D)
index.add(xb)

In [14]:
%%time
D, I = index.search(xq, k)

CPU times: user 29.5 ms, sys: 5.58 ms, total: 35.1 ms
Wall time: 35.2 ms


In [15]:
# Save as baseline. This is our 100% correct recall set.
baseline = I[0].tolist()

In [16]:
# Measure index size
faiss.write_index(index, "flat_index.index")
flat_index_size = os.path.getsize("flat_index.index")
os.remove("flat_index.index")
humanize.naturalsize(flat_index_size)

'512.0 MB'

# LSH Index
Locality Sensitive Hashing

In [89]:
D = xb.shape[1]  # dimensionality of Sift1M data

nbits = d*8  # resolution of bucketed vectors
# initialize index and add vectors
lsh_index = faiss.IndexLSH(d, nbits)
lsh_index.add(xb)

In [90]:
%%time
D, I = lsh_index.search(xq, k)

CPU times: user 5.1 ms, sys: 2.59 ms, total: 7.69 ms
Wall time: 6.22 ms


In [91]:
# Calculate Recall Percentage:
np.array(baseline)[np.in1d(baseline, I).tolist()].size / np.array(baseline).size * 100

100.0

In [92]:
# Measure index size
faiss.write_index(lsh_index, "lsh_index.index")
lsh_index_size = os.path.getsize("lsh_index.index")
os.remove("lsh_index.index")
humanize.naturalsize(lsh_index_size)

'128.5 MB'

# HNSW Index
Hierarchical Navigable Small World

In [17]:
%%time
D = xb.shape[1]  # dimensionality of Sift1M data

# set HNSW index parameters
M = 64  # number of connections each vertex will have
ef_search = 32  # depth of layers explored during search
ef_construction = 64  # depth of layers explored during index construction

# initialize index (d == 128)
hnsw_index = faiss.IndexHNSWFlat(D, M)
# set efConstruction and efSearch parameters
hnsw_index.hnsw.efConstruction = ef_construction
hnsw_index.hnsw.efSearch = ef_search
# add data to index
hnsw_index.add(xb)

CPU times: user 6min 55s, sys: 1.6 s, total: 6min 56s
Wall time: 46.3 s


In [21]:
%%timeit
D, I = hnsw_index.search(xq, k)

223 µs ± 7.05 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [19]:
# Calculate Recall Percentage:
np.array(baseline)[np.in1d(baseline, I).tolist()].size / np.array(baseline).size * 100

100.0

In [20]:
# Measure index size
faiss.write_index(hnsw_index, "hnsw_index.index")
hnsw_index_size = os.path.getsize("hnsw_index.index")
os.remove("hnsw_index.index")
humanize.naturalsize(hnsw_index_size)

'1.0 GB'

# IVFFlat Index
Plain Inverted File Index

In [97]:
%%time
nlist = 128  # number of cells/clusters to partition data into
D = xb.shape[1]  # dimensionality of Sift1M data

quantizer = faiss.IndexFlatIP(d)  # how the vectors will be stored/compared
ivf_index = faiss.IndexIVFFlat(quantizer, d, nlist)
ivf_index.train(xb)  # we must train the index to cluster into cells
ivf_index.add(xb)

ivf_index.nprobe = 4  # set how many of nearest cells to search

CPU times: user 520 ms, sys: 173 ms, total: 693 ms
Wall time: 269 ms


In [98]:
%%timeit
dist, I = ivf_index.search(xq, k)

975 µs ± 428 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [99]:
# Calculate Recall Percentage:
np.array(baseline)[np.in1d(baseline, I).tolist()].size / np.array(baseline).size * 100

80.0

In [100]:
# Measure index size
faiss.write_index(ivf_index, "ivf_index.index")
ivf_index_size = os.path.getsize("ivf_index.index")
os.remove("ivf_index.index")
humanize.naturalsize(ivf_index_size)

'520.1 MB'

## PQ Index
Product Quantizer

In [101]:
%%time
D = xb.shape[1] # dimensions of the vector
m = 8 # number of sub vectors to split
assert D % m == 0 # dimensions must dividable by the number of sub vectors

nbits = 8  # number of bits per subquantizer, k* = 2**nbits
pq_index = faiss.IndexPQ(D, m, nbits)
pq_index.train(xb) 
pq_index.add(xb)

CPU times: user 8.9 s, sys: 2.14 s, total: 11 s
Wall time: 4.63 s


In [102]:
%%timeit
dist, I = ivf_index.search(xq, k)

1.02 ms ± 22.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [103]:
# Calculate Recall Percentage:
np.array(baseline)[np.in1d(baseline, I).tolist()].size / np.array(baseline).size * 100

80.0

In [104]:
# Measure index size
faiss.write_index(pq_index, "pq_index.index")
pq_index_size = os.path.getsize("pq_index.index")
os.remove("pq_index.index")
humanize.naturalsize(pq_index_size)

'8.1 MB'

## Compound IVF + PQ Index

In [119]:
%%time
nlist = 128  # number of cells/clusters to partition data into
nbits = 8  # when using IVF+PQ, higher nbits values are not supported
D = xb.shape[1] # dimensions of the vector
m = 8 # number of sub vectors to split
assert D % m == 0 # dimensions must dividable by the number of sub vectors

quantizer = faiss.IndexFlatIP(d)  # how the vectors will be stored/compared
ivfpq_index = faiss.IndexIVFPQ(quantizer, D, nlist, m, nbits)
ivfpq_index.train(xb)  # we must train the index to cluster into cells
ivfpq_index.add(xb)

ivfpq_index.nprobe = 4  # set how many of nearest cells to search

CPU times: user 8.59 s, sys: 2.04 s, total: 10.6 s
Wall time: 4.64 s


In [120]:
%%timeit
dist, I = ivfpq_index.search(xq, k)

272 µs ± 276 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [121]:
# Calculate Recall Percentage:
np.array(baseline)[np.in1d(baseline, I).tolist()].size / np.array(baseline).size * 100

80.0

In [122]:
# Measure index size
faiss.write_index(ivfpq_index, "ivfpq_index.index")
ivfpq_index_size = os.path.getsize("ivfpq_index.index")
os.remove("ivfpq_index.index")
humanize.naturalsize(ivfpq_index_size)

'16.2 MB'